<a href="https://colab.research.google.com/github/chizuchizu/IOAI/blob/main/Task2/chizu_009_task2_complete_bs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ====================================================
# CFG
# ====================================================

class CFG:
    num_workers=4
    project = "IOAI_Task2_classification"
    name = "chizu_009_task2_complete_bs"

    # pseudo_base_model_name = "ioai2024japan/redrock_015_task2_finetune"
    base_model_name = "google-bert/bert-base-multilingual-uncased"
    base_tokenizer_name = "google-bert/bert-base-multilingual-uncased"

    # None -> training, otherwise -> load the model
    # pretrained_model_name = None # "ioai2024japan/redrock_005_task2_pretrain_wandb"
    pretrained_model_name = None # "/content/chizu_004_task2_complete_5e-5_pretrain"
    # tokenizer_name = "google-bert/bert-base-multilingual-uncased"
    num_classes = 5

    # training
    pretrain_epochs = 2
    classification_epochs = 30
    mlm_probability = 0.15

    scheduler='linear' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']

    lr = 5e-5

    # dataset
    max_length = 256

    train_batch_size = 64
    eval_batch_size = 64

    seed=42
    train=True

    pseudo_size = 60000
    pseudo_select_size = 1500

    if_wandb = True

# for wandb
cfg = dict(vars(CFG))
cfg = {k: v for k, v in cfg.items() if "__" not in k}

In [ ]:
from google.colab import userdata

read_access_token = userdata.get('hf_read')
write_access_token = userdata.get('hf_write')

import importlib
import torch, transformers

if '2.3.0' not in torch.__version__:
  !pip install torch==2.3.0
if transformers.__version__!='4.41.2':
  !pip install transformers==4.41.2

if importlib.util.find_spec('datasets') is None:
  !pip install datasets==2.18.0
  !pip install evaluate==0.4.2
  !pip install accelerate -U

if importlib.util.find_spec('wandb') is None:
  !pip install wandb -q

import os
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.functional import F
import torch.cuda.amp as amp

import torchvision
from torchvision import datasets, transforms, models

from tqdm.auto import tqdm
from transformers import DataCollatorForLanguageModeling

from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, get_scheduler, BertForMaskedLM, BertTokenizer

from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets

import evaluate

import wandb

from joblib import Parallel, delayed

from huggingface_hub import login

wandb.login(key=userdata.get('wandb_token'))
login(token=read_access_token)

brahmi_to_devanagari = {
    '𑀓': 'क', '𑀔': 'ख', '𑀕': 'ग', '𑀖': 'घ', '𑀗': 'ङ', '𑀘': 'च', '𑀙': 'छ',
    '𑀚': 'ज', '𑀛': 'झ', '𑀜': 'ञ', '𑀝': 'ट', '𑀞': 'ठ', '𑀟': 'ड', '𑀠': 'ढ',
    '𑀡': 'ण', '𑀢': 'त', '𑀣': 'थ', '𑀤': 'द', '𑀥': 'ध', '𑀦': 'न', '𑀧': 'प',
    '𑀨': 'फ', '𑀩': 'ब', '𑀪': 'भ', '𑀫': 'म', '𑀬': 'य', '𑀭': 'र', '𑀮': 'ल',
    '𑀯': 'व', '𑀰': 'श', '𑀱': 'ष', '𑀲': 'स', '𑀳': 'ह', '𑁦':'०', '𑁣': '90'
}

def transliterate_brahmi_to_devanagari(text):
    transliterated_text = ''
    for char in text:
        if char in brahmi_to_devanagari:
            transliterated_text += brahmi_to_devanagari[char]
        else:
            transliterated_text += char
    return transliterated_text

f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1.compute(predictions=predictions, references=labels, average='macro')

def to_device(batch, device):
    output = {}
    for k, v in batch.items():
        try:
            output[k] = v.to(device)
        except:
            output[k] = v
    return output


In [ ]:
def train_tokenizer(raw_dataset):
    train_corpus = []

    num_cores = 8

    train_corpus = Parallel(n_jobs=num_cores)(
        delayed(transliterate_brahmi_to_devanagari)(text) for text in tqdm(raw_dataset['train']["text"])
    )

    base_tokenizer = AutoTokenizer.from_pretrained(CFG.base_tokenizer_name)

    tokenizer = base_tokenizer.train_new_from_iterator(train_corpus, base_tokenizer.vocab_size)

    return tokenizer

In [ ]:
def train_one_epoch(model, scheduler, train_loader, optimizer, fp16=False):
    model.train()
    running_loss = 0.0
    progress_bar = tqdm(train_loader, dynamic_ncols=True)
    scaler = torch.cuda.amp.GradScaler()

    for step, batch in enumerate(progress_bar):
        batch = to_device(batch, "cuda")

        if fp16:
            with amp.autocast():
                outputs = model(
                    input_ids=batch["input_ids"],
                    attention_mask=batch["attention_mask"],
                    token_type_ids=batch["token_type_ids"],
                    labels=batch["labels"],
                )
                loss = outputs.loss

            # Scale loss for fp16 training
            scaler.scale(loss).backward()

            # Optimizer step with gradient scaling
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            scheduler.step()
        else:
            outputs = model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                token_type_ids=batch["token_type_ids"],
                labels=batch["labels"],
            )
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        if CFG.if_wandb:
            wandb.log(
                {
                    "train_loss": loss,
                    "lr": optimizer.param_groups[0]["lr"],
                    "step": step,
                }
            )

        text = f"step {step}, loss: {loss:.5f}"
        progress_bar.set_description(text)

def evaluate_model(model, eval_loader):
    model.eval()
    predictions = []
    labels = []
    for batch in eval_loader:
        batch = to_device(batch, "cuda")
        with torch.no_grad():
            outputs = model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                token_type_ids=batch["token_type_ids"],
                labels=batch["labels"],
            )

        logits = outputs.logits
        prediction = torch.argmax(logits, dim=-1)
        predictions.append(prediction.cpu().numpy())
        labels.append(batch["labels"].cpu().numpy())

    predictions = np.concatenate(predictions)
    labels = np.concatenate(labels)
    f1_score = f1.compute(predictions=predictions, references=labels, average='macro')
    return f1_score, predictions

def test_model(model, eval_loader):
    model.eval()
    predictions = []
    labels = []
    for batch in eval_loader:
        batch = to_device(batch, "cuda")
        with torch.no_grad():
            outputs = model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                token_type_ids=batch["token_type_ids"],
            )

        logits = outputs.logits
        prediction = torch.argmax(logits, dim=-1)
        predictions.append(prediction.cpu().numpy())

    predictions = np.concatenate(predictions)
    return predictions

In [ ]:
def pretrain(raw_dataset, tokenizer, transform_raw, fp16=True):
    print("=== Pretrain ===")

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=True,
        mlm_probability=CFG.mlm_probability
    )

    tokenized_data = raw_dataset.with_transform(transform_raw)

    train_dataset = tokenized_data["train"]

    train_loader = DataLoader(
        train_dataset,
        batch_size=CFG.train_batch_size,
        num_workers=0,
        pin_memory=True,
        shuffle=True,
        drop_last=True,
        collate_fn=data_collator,
    )

    model = BertForMaskedLM.from_pretrained(
        CFG.base_model_name
    ).cuda()

    num_training_steps = CFG.pretrain_epochs * len(train_loader)
    optimizer = optim.AdamW(model.parameters(), lr=CFG.lr)
    scheduler = get_scheduler(name=CFG.scheduler, optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

    for i in range(CFG.pretrain_epochs):
        train_one_epoch(model, scheduler, train_loader, optimizer, fp16)
        print(f'Epoch {i+1}')

    model_path = f"{CFG.name}_pretrain"
    model.save_pretrained(model_path)
    return model_path

In [ ]:
def finetine(base_model, train_dataset, eval_dataset, device, fp16=False):
    print("=== Finetune ===")
    model = AutoModelForSequenceClassification.from_pretrained(
        base_model, num_labels=CFG.num_classes
    ).cuda()

    train_loader = DataLoader(
        train_dataset,
        batch_size=CFG.train_batch_size,
        num_workers=0,
        pin_memory=True,
        shuffle=True,
        drop_last=True,
    )

    eval_loader = DataLoader(
        eval_dataset,
        batch_size=CFG.eval_batch_size,
        num_workers=0,
        pin_memory=True,
        shuffle=False,
        drop_last=False,
    )

    num_training_steps = CFG.classification_epochs * len(train_loader)
    optimizer = optim.AdamW(model.parameters(), lr=CFG.lr)
    scheduler = get_scheduler(name=CFG.scheduler, optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

    best_f1 = 0.0
    best_model_path = None

    model.to(device)
    for i in range(CFG.classification_epochs):
        train_one_epoch(model, scheduler, train_loader, optimizer)
        f1_score, _ = evaluate_model(model, eval_loader)
        f1_score = f1_score["f1"]

        if f1_score > best_f1:
            best_f1 = f1_score
            best_model_path = f"{CFG.name}_finetune_epoch_{i+1}"
            model.save_pretrained(best_model_path)


        if CFG.if_wandb:
            wandb.log(
                {
                    "epoch": i+1,
                    "f1": f1_score
                }
            )
        print(f'Epoch {i+1} {f1_score}')

    # model_path = f"{CFG.name}_finetune"
    # model.save_pretrained(model_path)
    return best_model_path

In [ ]:
def up_to_hub(model_name, model_path, tokenizer):
    model = AutoModelForSequenceClassification.from_pretrained(
        model_path, num_labels=CFG.num_classes
    )
    model.push_to_hub(
        f"ioai2024japan/{model_name}",
        token=write_access_token, private=True
    )
    tokenizer.push_to_hub(
        f"ioai2024japan/{model_name}",
        token=write_access_token, private=True
    )

In [ ]:
def main():



    raw_dataset = load_dataset('InternationalOlympiadAI/NLP_problem_raw', token=read_access_token)
    classification_dataset = load_dataset('InternationalOlympiadAI/NLP_problem', token=read_access_token)

    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    # Train tokenizer to Lang X
    tokenizer = train_tokenizer(raw_dataset)

    # for raw set
    def transform_raw(example_batch):
        example_batch["text"] = [transliterate_brahmi_to_devanagari(x) for x in example_batch["text"]]
        inputs = tokenizer([x for x in example_batch["text"]],  truncation=True, max_length=CFG.max_length, padding="max_length", return_tensors="pt")
        return inputs

    # for problem set
    def transform(example_batch):
        example_batch["text"] = [transliterate_brahmi_to_devanagari(x) for x in example_batch["text"]]
        inputs = tokenizer([x for x in example_batch["text"]],  truncation=True, max_length=CFG.max_length, padding="max_length", return_tensors="pt")
        inputs["labels"] = example_batch["label"]
        return inputs

    tokenized_data = classification_dataset.with_transform(transform)

    train_dataset = tokenized_data["train"]
    eval_dataset = tokenized_data["dev"]

    # Continual Pre-Training of MLM
    if CFG.pretrained_model_name is None:
        if CFG.if_wandb:
            wandb.init(
                name=CFG.name,
                project="IOAI_Task2_pretrain",
                config=cfg
            )
        pretrained_model_path = pretrain(raw_dataset, tokenizer, transform_raw, fp16=True)
        up_to_hub(f"{CFG.name}_pretrain", pretrained_model_path, tokenizer)
    else:
        pretrained_model_path = CFG.pretrained_model_name

    if CFG.if_wandb:
        wandb.init(
            name=CFG.name,
            project="IOAI_Task2_finetune",
            config=cfg
        )

    # Finetune with normal dataset
    finetuned_model_path = finetine(pretrained_model_path, train_dataset, eval_dataset, device)

    if CFG.if_wandb:
        wandb.finish()

    # # Get pseudo label
    # pseudo_data, confidences = pseudo_get_data(raw_dataset, transform_raw, finetuned_model_path, device)
    # pseudo_labeled_tokens = pseudo_data.with_transform(transform)

    # combined_train_dataset = concatenate_datasets([pseudo_labeled_tokens, train_dataset])

    # # Finetune with pseudo dataset and normal dataset
    # final_model_name = finetine(pretrained_model_path, combined_train_dataset, eval_dataset, device)

    return finetuned_model_path, tokenizer

In [ ]:
final_model_name, tokenizer = main()

In [ ]:
up_to_hub(CFG.name, final_model_name, tokenizer)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    final_model_name, num_labels=CFG.num_classes
)

In [ ]:
# run the trained model on a dev/test split
classification_dataset = load_dataset('InternationalOlympiadAI/NLP_problem', token=read_access_token)

def transform_raw(example_batch):
    example_batch["text"] = [transliterate_brahmi_to_devanagari(x) for x in example_batch["text"]]
    inputs = tokenizer([x for x in example_batch["text"]],  truncation=True, max_length=CFG.max_length, padding="max_length", return_tensors="pt")
    return inputs

data_split = "dev"
tokenized_data = classification_dataset.with_transform(transform_raw)
test_dataset = tokenized_data[data_split]

test_loader = DataLoader(
    test_dataset,
    batch_size=CFG.eval_batch_size,
    num_workers=0,
    pin_memory=True,
    shuffle=False,
    drop_last=False,
)
model.cuda()

In [ ]:
predictions = test_model(model, test_loader)

In [ ]:
# write the predictions to a file
with open('{}_predictions.txt'.format(data_split), 'w') as outfile:
  outfile.write('\n'.join([str(p) for p in predictions.tolist()]))

In [ ]:
predictions

In [ ]:
def terminate_session():
    # Terminate this session

    from google.colab import runtime
    runtime.unassign()

terminate_session()